In [5]:
import pyro
import torch
import numpy as np
import pyro.poutine as poutine
import pyro.distributions as dist

pyro.set_rng_seed(101)

def walk_model():
    import pyro
    import torch

    start = pyro.sample("start", pyro.distributions.Uniform(0, 3))
    t = 0
    position = start
    distance = torch.tensor(0.0)
    while position > 0 and position < 10:
        step = pyro.sample(f"step_{t}", pyro.distributions.Uniform(-1, 1))
        distance = distance + torch.abs(step)
        position = position + step
        t = t + 1
    pyro.sample("obs", pyro.distributions.Normal(1.1, 0.1), obs=distance)
    return start.item()

def make_log_joint(model):
    def _log_joint(cond_data, *args, **kwargs):
        conditioned_model = poutine.condition(model, data=cond_data)
        trace = poutine.trace(conditioned_model).get_trace(*args, **kwargs)
        # print(dict(trace)) 
        return trace.log_prob_sum()
    return _log_joint

scale_log_joint = make_log_joint(walk_model)
value = scale_log_joint({"start": torch.tensor(0.5), "step_0": torch.tensor(-0.4), "step_1": torch.tensor(-0.9), "obs": torch.tensor(1.3)})

from scipy.stats import norm



print(np.log(norm(1.1, 0.1).pdf(1.3) * 1/2 * 1/2 * 1/3))
print(value)

-3.1012600899986262
tensor(-3.1013)


In [ ]:
def _potential_fn(self, params):
        params_constrained = {k: self.transforms[k].inv(v) for k, v in params.items()}
        cond_model = poutine.condition(self.model, params_constrained)
        model_trace = poutine.trace(cond_model).get_trace(
            *self.model_args, **self.model_kwargs
        )
        log_joint = self.trace_prob_evaluator.log_prob(model_trace)
        for name, t in self.transforms.items():
            log_joint = log_joint - torch.sum(
                t.log_abs_det_jacobian(params_constrained[name], params[name])
            )
        return -log_joint

